In [2]:
import numpy as np
import pandas as pd
import ast
import csv

from ipywidgets import interact
import ipywidgets as widgets

from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import plot, iplot, init_notebook_mode

init_notebook_mode(connected=True)

import folium

#Calculate centroids
def centeroidnp(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    return np.array([sum_y/length, sum_x/length])

In [64]:
from ipywidgets import interactive, HBox, VBox, widgets
from ipywidgets.embed import embed_minimal_html

def mammals_countries(mammal):    
    #Load and prepare data
    df = pd.read_csv('mammals_country_counts_old.csv')
    npages = df.loc[df['species']==mammal]['number_results'].values[0]
    aux = df.loc[df['species']==mammal]['country_counts'].values[0]
    aux = aux.strip('[]')
    aux_all = list(ast.literal_eval(aux))

    #Check if it's a valid country name and counts occurrences
    with open('countries_names.csv', 'r') as f:
        reader = csv.reader(f)
        valid_countries = list(reader)[0]
    country_names = []
    country_counts = []
    for i in aux_all:   
        if i[0].title() in valid_countries:  
            country_names.append(i[0].title())
            country_counts.append(i[1])
    country_names = np.array(country_names)
    country_counts = 100*np.array(country_counts)/npages
    
    return country_names, country_counts



def bar_graph_mammals(mammal, cnt_threshold=5): 
    #Get countries counts
    country_names, country_counts = mammals_countries(mammal)
    if len(country_names) > 30:
        country_names = country_names[0:30]
        country_counts = country_counts[0:30]
    
    #Make Plotly graphics
    trace0 = go.Bar(
        x=country_names,
        y=country_counts,
        marker=dict( color='rgb(158,202,225)',
                     line=dict(color='rgb(8,48,107)', width=1.5) ),
        opacity=0.6 )

    data = [trace0]
    
    #Slider
    #sliders = [dict(
    #    active = 10,
    #    currentvalue = {"prefix": "Frequency: "},
    #    pad = {"t": 50},
    #    #steps = steps 
    #)]
   
    
    layout = dict( height=370, width=900, xaxis=dict(tickangle=-45), title=mammal+' presence',
                   #sliders=sliders,
                   yaxis=dict( title='Occurrences on web pages [%]' ),
                   paper_bgcolor='rgba(0,0,0,0)',
                   plot_bgcolor='rgba(0,0,0,0)',
                   shapes=[{
                        'type': 'line',
                        'x0': 0,
                        'y0': cnt_threshold,
                        'x1': len(country_names),
                        'y1': cnt_threshold,
                        'line': {
                            'color': 'rgb(0, 0, 0)',
                            'width': 1,
                            'dash': 'dot'}}] )

    fig = go.Figure(data=data, layout=layout)
    
    #Save bar plots
    iplot(fig)
    plot(fig, filename='media/bar_graph_'+mammal+'.html')
    
    # create FigureWidget from fig
    f = go.FigureWidget(data=fig.data, layout=fig.layout)

    slider = widgets.IntSlider(
        min=5,
        max=50,
        step=1,
        readout=True,
        orientation='vertical',
        readout_format='.1f',
        description='reference')
    #slider.layout.width = '800px'
    
    #function that will modify the reference line
    def update_ref_line(y):
        f.layout.shapes[0].y0 = y
        f.layout.shapes[0].y1 = y      
    


    # display the FigureWidget and slider with center justification
    #vb = VBox( [interactive(update_ref_line, y=slider), f] )
    #left_box = VBox([interactive(update_ref_line, y=slider)])
    #right_box = VBox([f])
    #hb = HBox([left_box, right_box])
    #hb.layout.align_items = 'center'
    #display(hb)
    
    return country_names, country_counts
    

mammal = 'koala'    
cnt_threshold = 20
country_names, country_counts = bar_graph_mammals(mammal, cnt_threshold = cnt_threshold) 

#mammal_list = ['hedgehog', 'lion', 'wolf', 'fox', 'zebra', 'giraffe',
#               'bat', 'sloth', 'capybara', 'elephant', 'rhino',
#               'hippo', 'tiger', 'panda', 'kangaroo', 'koala', 'human']

#for mammal in mammal_list[12:13]:    
#        bar_graph_mammals(mammal)
        

In [65]:
#Map ---------------------------------------------------------------------------------------------------------
# Make an empty map
my_map = folium.Map(location=[20, 0], tiles="Mapbox Bright", zoom_start=2)
#my_map = folium.Map(location=[20, 0], tiles="Stamen Terrain", zoom_start=2)

#Countries layer
#countries_geo = 'countries_geo.json'
countries_geo = 'world-countries.json'

countries_df = pd.read_json(countries_geo)
for index, row in countries_df.iterrows():
    countries_df.at[index,'country'] = row['features']['properties']['name']

centroids = pd.read_csv('centroids.csv')  #centroid marks to print icons


chosen_indexes = country_counts > cnt_threshold

chosen_data = pd.DataFrame(data=country_names[chosen_indexes], columns=['country'])
chosen_data['quantity'] = country_counts[chosen_indexes]
#chosen_data['species'] = mammal

#Data
#species_data = pd.read_csv('mammals_countries_byhand.csv')
#species_grp = species_data.groupby('species')
#chosen_data = species_grp.get_group(mammal)


# Add the color for the chloropleth:
my_map.choropleth(
    geo_data=countries_geo,
    name='choropleth',
    data=chosen_data,
    columns=['country', 'quantity'],
    key_on='properties.name', #'feature.id',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color ='#ffffff00',
    legend_name='Estimated presence of '+mammal+'s',
)
folium.LayerControl().add_to(my_map)


#create mammal markers and add them to map object

#if mammal == 'hedgehog':
#    for i in np.arange(0,30):
#        icon = folium.features.CustomIcon('./icons/icon_'+mammal+'.png', icon_size=(40,40))
#        popupIcon = "<strong>mammal</strong><br>Hedgehogs live in Africa, Asia and Europe"
#        lng = np.random.uniform(-10,180)
#        lat = np.random.uniform(-10,70)
#        folium.Marker([lat,lng], tooltip=mammal, popup=popupIcon, icon=icon).add_to(my_map)

#if mammal == 'bat':
#    for i in np.arange(0,20):
#        icon = folium.features.CustomIcon('./icons/icon_'+mammal+'.png', icon_size=(40,40))
#        popupIcon = "<strong>mammal</strong><br>Bats are everywhere!!"
#        lng = np.random.uniform(-180,180)
#        lat = np.random.uniform(-90,90)
#        folium.Marker([lat,lng], tooltip=mammal, popup=popupIcon, icon=icon).add_to(my_map)

if mammal == 'human':
    for i in np.arange(0,30):
        icon = folium.features.CustomIcon('./icons/icon_'+mammal+'.png', icon_size=(40,40))
        popupIcon = "<strong>mammal</strong><br>Humans are everywhere!!"
        lng = np.random.uniform(-180,180)
        lat = np.random.uniform(-90,90)
        folium.Marker([lat,lng], tooltip=mammal, popup=popupIcon, icon=icon).add_to(my_map)

else:    
    for index, row in chosen_data.iterrows():
        #create icons from images
        icon = folium.features.CustomIcon('./icons/icon_'+mammal+'.png', icon_size=(25,25))
        #create popup descriptions
        popupIcon = "<strong>mammal</strong><br>Population"
        if sum(centroids['country']==row.country) == 1:
            lat = centroids.loc[centroids['country']==row.country]['lat']
            lng = centroids.loc[centroids['country']==row.country]['lng']
            folium.Marker([lat.values[0],lng.values[0]], tooltip=mammal, popup=popupIcon, icon=icon).add_to(my_map)

# Save to html
my_map.save('media/map_'+mammal+'.html')
my_map

C:\Users\Luiz\Anaconda3\envs\tf_obj_detect\lib\site-packages\folium\folium.py:426: FutureWarning:

The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.



In [17]:
chosen_indexes

array([ True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False])

In [48]:
# Make an empty map
my_map = folium.Map(location=[20, 0], tiles="Mapbox Bright", zoom_start=2)
#my_map = folium.Map(location=[20, 0], tiles="Stamen Terrain", zoom_start=2)

#Countries layer
#countries_geo = 'countries_geo.json'
countries_geo = 'world-countries.json'

countries_df = pd.read_json(countries_geo)
for index, row in countries_df.iterrows():
    countries_df.at[index,'country'] = row['features']['properties']['name']
    
centroids = pd.read_csv('centroids.csv')    

cnt_threshold = 3
chosen_indexes = country_counts > cnt_threshold

chosen_data = pd.DataFrame(data=country_names[chosen_indexes], columns=['country'])
chosen_data['quantity'] = country_counts[chosen_indexes]
#chosen_data['species'] = mammal

#Data
#species_data = pd.read_csv('mammals_countries_byhand.csv')
#species_grp = species_data.groupby('species')
#chosen_data = species_grp.get_group(mammal)


# Add the color for the chloropleth:
my_map.choropleth(
    geo_data=countries_geo,
    name='choropleth',
    data=chosen_data,
    columns=['country', 'quantity'],
    key_on='properties.name', #'feature.id',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color ='#ffffff00',
    legend_name='Estimated presence of '+mammal+'s',
)
folium.LayerControl().add_to(my_map)


#create mammal markers and add them to map object

if mammal == 'hedgehog':
    for i in np.arange(0,30):
        icon = folium.features.CustomIcon('./icons/icon_'+mammal+'.png', icon_size=(40,40))
        popupIcon = "<strong>mammal</strong><br>Hedgehogs live in Africa, Asia and Europe"
        lng = np.random.uniform(-10,180)
        lat = np.random.uniform(-10,70)
        folium.Marker([lat,lng], tooltip=mammal, popup=popupIcon, icon=icon).add_to(my_map)
        
elif mammal == 'bat':
    for i in np.arange(0,20):
        icon = folium.features.CustomIcon('./icons/icon_'+mammal+'.png', icon_size=(40,40))
        popupIcon = "<strong>mammal</strong><br>Bats are everywhere!!"
        lng = np.random.uniform(-180,180)
        lat = np.random.uniform(-90,90)
        folium.Marker([lat,lng], tooltip=mammal, popup=popupIcon, icon=icon).add_to(my_map)

elif mammal == 'human':
    for i in np.arange(0,30):
        icon = folium.features.CustomIcon('./icons/icon_'+mammal+'.png', icon_size=(40,40))
        popupIcon = "<strong>mammal</strong><br>Humans are everywhere!!"
        lng = np.random.uniform(-180,180)
        lat = np.random.uniform(-90,90)
        folium.Marker([lat,lng], tooltip=mammal, popup=popupIcon, icon=icon).add_to(my_map)
        
else:    
    for index, row in chosen_data.iterrows():
        #create icons from images
        icon = folium.features.CustomIcon('./icons/icon_'+mammal+'.png', icon_size=(25,25))
        #create popup descriptions
        popupIcon = "<strong>mammal</strong><br>Population"
        if sum(centroids['country']==row.country) == 1:
            lat = centroids.loc[centroids['country']==row.country]['lat']
            lng = centroids.loc[centroids['country']==row.country]['lng']
            folium.Marker([lat.values[0],lng.values[0]], tooltip=mammal, popup=popupIcon, icon=icon).add_to(my_map)




        
#my_map

# Save to html
my_map.save('media/map_'+mammal+'.html')


C:\Users\Luiz\Anaconda3\envs\tf_obj_detect\lib\site-packages\folium\folium.py:426: FutureWarning:

The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.

